In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://www.w3schools.com/angular/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [3]:
print(status)

200


In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> Tutorial
h2 -> Examples
h2 -> Reference
h1 -> AngularJS Tutorial
h2 -> This Tutorial
h2 -> Try it Yourself Examples in Every Chapter
h3 -> AngularJS Example
h2 -> What You Should Already Know
h2 -> AngularJS History
h2 -> AngularJS Examples
h2 -> AngularJS Reference
h2 -> Report Error
h2 -> Thank You For Helping Us!


In [6]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left">Tutorial</h2>,
 <h2 class="left">Examples</h2>,
 <h2 class="left">Reference</h2>]

In [7]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

Tutorial
Examples
Reference


In [8]:
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

In [9]:
# getting the Headings, Sub-Headings, Links and Contents

# CREATING RELEVANT VARIABLES
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []
ordered_lists = []
list_headings = []
examples_per_page = []
excercises = []
base_url = "https://www.w3schools.com/angular/"
course_url = "https://www.w3schools.com/angular/"

# FINDING ALL THE LINKS PRESENT IN THE LEFT MENU AND ITERATING OVER THE LINKS TO SCRAPE THE DATA
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
           
            if extra is not None :
                print(extra)
                if extra[0] == '/':
                    base_url = "https://www.w3schools.com"
                    
            elif extra is None :
                continue 
           
                
                
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            page_examples = soup_page.find_all('div', class_ = 'w3-code notranslate htmlHigh')
            
            ##################################################################################################
            
            # Examples per page
            pg_example = []
            for example in page_examples:
                nw = []
                for i in list(example):
                    if str(i) == '<br/>':
                        pass
                    else:
                        nw.append(i)
                pg_example.append(nw)
            examples_per_page.append(pg_example)
                
            ##################################################################################################
            
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            if len(row) !=0:
                if row[0] == '&':
                    row = row[3:]
            table_new.append(row)
            table_string = ''
            for each in table_new:
                table_string += each + '$$$'
            table_new = table_string[:-3]
            page_table.append(table_new)
            
            ##################################################################################################
            
            # Points for Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                unorder.append(start_string)
            if len(unorder) == 0:
                unorder = None
            else:
                unorder_string = ''
                for i in unorder:
                    unorder_string += i +'|||'
                unorder = unorder_string[:-3]
            unordered_lists.append(unorder)
            
                        
            ##################################################################################################
            #Points for ordered list.
            ol = soup_page.find_all('ol')
            order = []
            for i in ol:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                order.append(start_string)
            
            if len(order) == 0:
                order = None
            else:
                order_string = ''
                for i in order:
                    order_string += i +'|||'
                order = order_string[:-3]
            ordered_lists.append(order)
            
            ##################################################################################################
            
            # Headings and Subheadings and Description
            
            list_headings_per_page = []
            for page_data in page_content:
                for j in str(page_data).split('\n'):
                    list_headings_per_page.append(j)
                page_paragraphs = page_data.find_all('p')
        
            content_per_page = ''
            for i in page_paragraphs:
                content_per_page += i.text + '|||'
            content_per_page = content_per_page[:-3].replace('\n', '')
            sub_page_content_list.append(content_per_page)
            
            prev = ''
            ul_ol_heading = ''
            for i in list_headings_per_page:
                if ('ul' in i) and (('h2' in prev) or ('h3' in prev)):
                    ul_ol_heading += prev[4:-5] + '|||'
                prev = i
            list_headings.append(ul_ol_heading[:-3])
#             time.sleep(5)      

default.asp
angular_intro.asp
angular_expressions.asp
angular_modules.asp
angular_directives.asp
angular_model.asp
angular_databinding.asp
angular_controllers.asp
angular_scopes.asp
angular_filters.asp
angular_services.asp
angular_http.asp
angular_tables.asp
angular_select.asp
angular_sql.asp
angular_htmldom.asp
angular_events.asp
angular_forms.asp
angular_validation.asp
angular_api.asp
angular_w3css.asp
angular_includes.asp
angular_animations.asp
angular_routing.asp
angular_application.asp
angular_examples.asp
angular_ref_directives.asp


In [10]:
length = len(course_links)
length

27

In [11]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(list_headings), len(unordered_lists),len(ordered_lists),  len(page_table), len(examples_per_page)

(27, 27, 27, 27, 27, 27, 27, 27, 27)

In [12]:
from csv import writer

# writing the file
with open('angular_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('angular_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,Tutorial,AngularJS HOME,https://www.w3schools.com/angular/default.asp,AngularJS extends HTML with new attributes.|||...,This Tutorial|||Try it Yourself Examples in Ev...,HTML@@@CSS@@@JavaScript,NaN,NaN,"[['\n<!DOCTYPE html>', '\n<html lang=""en-US"">'..."
1,Tutorial,AngularJS Intro,https://www.w3schools.com/angular/angular_intr...,AngularJS is a JavaScript framework. It can be...,AngularJS is a JavaScript Framework|||AngularJ...,NaN,NaN,NaN,"[['\n<script\nsrc=""https://ajax.googleapis.com..."
2,Tutorial,AngularJS Expressions,https://www.w3schools.com/angular/angular_expr...,AngularJS binds data to HTML using Expressions...,AngularJS Expressions|||AngularJS Numbers|||An...,NaN,NaN,NaN,"[['\n<!DOCTYPE html>', '\n<html>', '<script sr..."
3,Tutorial,AngularJS Modules,https://www.w3schools.com/angular/angular_modu...,An AngularJS module defines an application.|||...,Creating a Module|||Adding a Directive|||Modul...,NaN,NaN,NaN,"[['\n <div ng-app=""myApp"">...</div>', '<scr..."
4,Tutorial,AngularJS Directives,https://www.w3schools.com/angular/angular_dire...,AngularJS lets you extend HTML with new attrib...,AngularJS Directives|||Data Binding|||Create N...,Provide type validation for application data (...,NaN,NaN,"[['\n<div ng-app="""" ng-init=""firstName=\'John\..."


In [13]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [14]:
from csv import writer

# writing the file
with open('angular_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('angular_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,Tutorial,AngularJS HOME,https://www.w3schools.com/angular/default.asp,AngularJS extends HTML with new attributes.|||...,This Tutorial|||Try it Yourself Examples in Ev...,HTML@@@CSS@@@JavaScript,NaN,NaN,"[['\n<!DOCTYPE html>', '\n<html lang=""en-US"">'..."
1,Tutorial,AngularJS Intro,https://www.w3schools.com/angular/angular_intr...,AngularJS is a JavaScript framework. It can be...,AngularJS is a JavaScript Framework|||AngularJ...,NaN,NaN,NaN,"[['\n<script\nsrc=""https://ajax.googleapis.com..."
2,Tutorial,AngularJS Expressions,https://www.w3schools.com/angular/angular_expr...,AngularJS binds data to HTML using Expressions...,AngularJS Expressions|||AngularJS Numbers|||An...,NaN,NaN,NaN,"[['\n<!DOCTYPE html>', '\n<html>', '<script sr..."
3,Tutorial,AngularJS Modules,https://www.w3schools.com/angular/angular_modu...,An AngularJS module defines an application.|||...,Creating a Module|||Adding a Directive|||Modul...,NaN,NaN,NaN,"[['\n <div ng-app=""myApp"">...</div>', '<scr..."
4,Tutorial,AngularJS Directives,https://www.w3schools.com/angular/angular_dire...,AngularJS lets you extend HTML with new attrib...,AngularJS Directives|||Data Binding|||Create N...,Provide type validation for application data (...,NaN,NaN,"[['\n<div ng-app="""" ng-init=""firstName=\'John\..."


In [15]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [16]:
dataset = dataset.replace(np.nan, '', regex=True)

In [17]:
# Making a separate column for those heading which are not associated with the Lists
new_col = [None for i in range(len(course_links))]
dataset['Other Headings'] = pd.Series(new_col)
for i in range(len(dataset['Unordered Lists'])):
    if dataset['Unordered Lists'][i] == '':
        dataset['Other Headings'][i] = dataset['List Headings'][i]
        dataset['List Headings'][i] = ''

In [18]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)
dataset.head(10)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples,Other Headings
0,Tutorial,AngularJS HOME,https://www.w3schools.com/angular/default.asp,AngularJS extends HTML with new attributes.|||...,This Tutorial|||Try it Yourself Examples in Ev...,HTML@@@CSS@@@JavaScript,,,"[['\n<!DOCTYPE html>', '\n<html lang=""en-US"">'...",
1,Tutorial,AngularJS Intro,https://www.w3schools.com/angular/angular_intr...,AngularJS is a JavaScript framework. It can be...,,,,,"[['\n<script\nsrc=""https://ajax.googleapis.com...",AngularJS is a JavaScript Framework|||AngularJ...
2,Tutorial,AngularJS Expressions,https://www.w3schools.com/angular/angular_expr...,AngularJS binds data to HTML using Expressions...,,,,,"[['\n<!DOCTYPE html>', '\n<html>', '<script sr...",AngularJS Expressions|||AngularJS Numbers|||An...
3,Tutorial,AngularJS Modules,https://www.w3schools.com/angular/angular_modu...,An AngularJS module defines an application.|||...,,,,,"[['\n <div ng-app=""myApp"">...</div>', '<scr...",Creating a Module|||Adding a Directive|||Modul...
4,Tutorial,AngularJS Directives,https://www.w3schools.com/angular/angular_dire...,AngularJS lets you extend HTML with new attrib...,AngularJS Directives|||Data Binding|||Create N...,Provide type validation for application data (...,,,"[['\n<div ng-app="""" ng-init=""firstName=\'John\...",
5,Tutorial,AngularJS Model,https://www.w3schools.com/angular/angular_mode...,The ng-model directive binds the value of HTML...,,ng-empty@@@ng-not-empty@@@ng-touched@@@ng-unto...,,,"[['\n <div ng-app=""myApp"" ng-controller=""my...",
6,Tutorial,AngularJS Data Binding,https://www.w3schools.com/angular/angular_data...,Data binding in AngularJS is the synchronizati...,,,,,"[['\n <p ng-bind=""firstname""></p>'], ['\n ...",Data Model|||HTML View|||Two-way Binding|||Ang...
7,Tutorial,AngularJS Controllers,https://www.w3schools.com/angular/angular_cont...,AngularJS controllers control the data of Angu...,,,,,"[['\n <div ng-app=""myApp"" ng-controller=""my...",AngularJS Controllers
8,Tutorial,AngularJS Scopes,https://www.w3schools.com/angular/angular_scop...,The scope is the binding part between the HTML...,How to Use the Scope?|||Understanding the Scope,"View, which is the HTML.@@@Model, which is the...",,,[],
9,Tutorial,AngularJS Filters,https://www.w3schools.com/angular/angular_filt...,Filters can be added in AngularJS to format da...,AngularJS Filters,currency Format a number to a currency format....,,{{x.name}}|||{{x.country}},"[['\n<div ng-app=""myApp"" ng-controller=""person...",
